In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.listdir('/kaggle/input')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image

In [ ]:
def load_imgs(impath):
    imgs=[]
    label=[]
    l1=os.listdir(impath)
    for i in l1:
        l2=os.listdir(impath+'/'+i)
        for j in l2:
            c=0
            img=Image.open(impath+i+'/'+j)
            img=img.resize(size=(28,28))
            #img=img.convert('L')
            img=np.array(img,dtype=np.float16)/255
            imgs.append(np.array(img)) #.flatten())
            label.append(i)
            del img
        
        '''if(c%1000==0):
            plt.imshow(img)
            plt.show()
            c=c+1'''
    return np.array(imgs),label

In [ ]:
x,y=load_imgs('/kaggle/input/eurosat-dataset/EuroSATallBands/SeaLake/')
x.shape,len(y)

In [ ]:
x

In [ ]:
y

In [ ]:
target=pd.Series(y,dtype='category')
target

In [ ]:
target.value_counts()

In [ ]:
t=target.cat.codes
t

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,t,test_size=0.2,shuffle=True)
train_x.shape,test_x.shape,train_y.shape,test_y.shape

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential

In [ ]:
cmodel=Sequential()
cmodel.add(Conv2D(32,activation='relu', kernel_size=3, input_shape=(train_x.shape[1:])))
cmodel.add(Conv2D(64,activation='relu', kernel_size=3,))
cmodel.add(Conv2D(128,activation='relu', kernel_size=3, ))
cmodel.add(Conv2D(256,activation='relu', kernel_size=3, ))

cmodel.add(Flatten())
#cmodel.add(Dense(500,activation='softmax'))

cmodel.add(Dense(3,activation='softmax'))
cmodel.summary()

In [ ]:
cmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cmodel.fit(train_x,train_y,epochs=30)

In [ ]:
yhat=np.argmax(cmodel.predict(test_x), axis=-1)
yhat

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y,yhat)

In [ ]:
cmodel=Sequential()
cmodel.add(Conv2D(32,activation='relu', kernel_size=3, input_shape=(train_x.shape[1:])))
cmodel.add(MaxPooling2D((2,2)))
cmodel.add(Conv2D(64,activation='relu', kernel_size=3,))
cmodel.add(MaxPooling2D((2,2)))
#cmodel.add(Conv2D(128,activation='relu', kernel_size=3, ))
#cmodel.add(MaxPooling2D((2,2)))
#cmodel.add(Conv2D(256,activation='relu', kernel_size=3, ))
#cmodel.add(MaxPooling2D((2,2)))

cmodel.add(Flatten())
#cmodel.add(Dense(500,activation='softmax'))

cmodel.add(Dense(3,activation='softmax'))
cmodel.summary()

In [ ]:
cmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cmodel.fit(train_x,train_y,epochs=40)

In [ ]:
yhat=np.argmax(cmodel.predict(test_x), axis=-1)

In [ ]:
accuracy_score(test_y,yhat)

In [ ]:
plt.imshow(np.uint8(train_x[0]*255))

In [ ]:
plt.figure()

#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(2,2) 

# use the created array to output your multiple images. In this case I have stacked 4 images vertically
axarr[0,0].imshow(np.uint8(train_x[0]*255))
axarr[0,1].imshow(np.uint8(train_x[1]*255))
axarr[1,0].imshow(np.uint8(train_x[2]*255))
axarr[1,1].imshow(np.uint8(train_x[3]*255))

In [ ]:
cmodel.layers[0].output

In [ ]:
outputs = [layer.output for layer in cmodel.layers]
outputs

In [ ]:
cmodel.layers[0].output

In [ ]:
from tensorflow.keras import backend as K

outputs = [K.function([cmodel.input], [layer.output])(train_x) for layer in cmodel.layers]
outputs 

In [ ]:
my_img=outputs[0,0,:,:0]
my_img.shape

In [ ]:
plt.imshow(my_img)

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

n=8
m=outputs.shape[-1]//n

fig = plt.figure(figsize=(20., 20.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(m, n),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )
l=[]
for i in range(m):
    for j in range(n):
        l.append(outputs[0,0,:,:,i+j]*255)
for ax, im in zip(grid, l):
    # Iterating over the grid returns the Axes.
    ax.imshow(im,cmap='gray')

plt.show()